In [270]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sklearn.preprocessing

In [271]:
data = pd.read_csv("data.csv")
edit = data.copy()

C:\Users\ssola\AppData\Local\Temp\ipykernel_8588\492614709.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data.csv")


In [272]:
data.head(5)

,nombre_inmueble,aviso,precio,superficie,tipo,dormitorios,direccion,barrio,negocio,fecha
0,CASAS-ALQUILER-TEMPORAL-PUNTA-COLORADA-034,16579724,U$S 1.750,600 Mts,Casa,2 dormitorios,Av. Primera entre Rambla Costanera y 8 Guazubi...,Punta Colorada,Alquiler temporario,2021-01-03 00:00:00
1,"Alquilo Apto. 2 Dorm. Vista , 2 da Enero y Mar...",16735528,$U 800,NaN,Apartamento,2 dormitorios,Mi hijo el doctor y En Familia,Punta Del Este,Alquiler temporario,2021-01-03 00:00:00
2,"Casa en Cerro del Burro ""MIRADOR II""",13795860,U$S 100.000,300 Mts,Casa,1 dormitorio,... y Cerro del burro,Piriápolis,Venta,2021-01-03 00:00:00
3,Oficina Alquiler World Trade Center -WTC- Poci...,15655688,U$S 1.600,NaN,Oficina,NaN,... y Buceo,Buceo,Alquiler,2021-01-03 00:00:00
4,Terreno en Punta Colorada,12805785,U$S 67.000,615 Mts,Terreno,NaN,TE201282 y Punta colorada,Punta Colorada,Venta,2021-01-03 00:00:00


In [273]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96583 entries, 0 to 96582
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nombre_inmueble  96462 non-null  object
 1   aviso            96583 non-null  object
 2   precio           96462 non-null  object
 3   superficie       81049 non-null  object
 4   tipo             96462 non-null  object
 5   dormitorios      75070 non-null  object
 6   direccion        96128 non-null  object
 7   barrio           96462 non-null  object
 8   negocio          96462 non-null  object
 9   fecha            96583 non-null  object
dtypes: object(10)
memory usage: 7.4+ MB


Clasificación por:
    - tipo de finca
    - tipo de negocio: afecta el precio (alquileres mensuales)

In [274]:
data["negocio"].value_counts()

Venta                  66165
Alquiler               15553
Alquiler temporario    14744
Name: negocio, dtype: int64

In [275]:
data["tipo"].value_counts()

Apartamento    45098
Casa           29972
Terreno        11076
Oficina         4455
Local           3944
Campo           1375
Comercio         202
Edificio         157
Garage           111
Piezas            66
Otro               6
Name: tipo, dtype: int64

In [276]:
data["direccion"].isna().sum(), data["direccion"].count() #Conteo de direcciones

(455, 96128)

In [277]:
data["barrio"].isna().sum(), data["barrio"].count() #Conteo de barrio, posibilidad de imputar direcciones faltantes

(121, 96462)

In [278]:
data["precio"].isna().sum()
#Si el estudio es sobre oportunidad de inversión, estos datos deben eliminarse

121

In [279]:
data["fecha"] #Irrelevante para el EDA, eliminable

0        2021-01-03 00:00:00
1        2021-01-03 00:00:00
2        2021-01-03 00:00:00
3        2021-01-03 00:00:00
4        2021-01-03 00:00:00
                ...         
96578    2021-12-07 00:00:00
96579    2021-12-07 00:00:00
96580    2021-12-07 00:00:00
96581    2021-12-07 00:00:00
96582    2021-12-07 00:00:00
Name: fecha, Length: 96583, dtype: object

In [280]:
data["aviso"] #Irrelevante para el modelo, eliminable provisoriamente

0        16579724
1        16735528
2        13795860
3        15655688
4        12805785
           ...   
96578    18610881
96579    17710617
96580    12804460
96581    18611010
96582    18610480
Name: aviso, Length: 96583, dtype: object

In [281]:
edit.drop("aviso",axis=1,inplace=True) #Eliminación de columna aviso

In [282]:
edit

,nombre_inmueble,precio,superficie,tipo,dormitorios,direccion,barrio,negocio,fecha
0,CASAS-ALQUILER-TEMPORAL-PUNTA-COLORADA-034,U$S 1.750,600 Mts,Casa,2 dormitorios,Av. Primera entre Rambla Costanera y 8 Guazubi...,Punta Colorada,Alquiler temporario,2021-01-03 00:00:00
1,"Alquilo Apto. 2 Dorm. Vista , 2 da Enero y Mar...",$U 800,NaN,Apartamento,2 dormitorios,Mi hijo el doctor y En Familia,Punta Del Este,Alquiler temporario,2021-01-03 00:00:00
2,"Casa en Cerro del Burro ""MIRADOR II""",U$S 100.000,300 Mts,Casa,1 dormitorio,... y Cerro del burro,Piriápolis,Venta,2021-01-03 00:00:00
3,Oficina Alquiler World Trade Center -WTC- Poci...,U$S 1.600,NaN,Oficina,NaN,... y Buceo,Buceo,Alquiler,2021-01-03 00:00:00
4,Terreno en Punta Colorada,U$S 67.000,615 Mts,Terreno,NaN,TE201282 y Punta colorada,Punta Colorada,Venta,2021-01-03 00:00:00
...,...,...,...,...,...,...,...,...,...
96578,LOCALES-COMERCIALES-ALQUILER-ANUAL-PIRIAPOLIS-...,$U 22.000,NaN,Local,NaN,Tucumán entre Sierra y Sanabria y,Piriápolis,Alquiler,2021-12-07 00:00:00
96579,"Casa en Punta Colorada ""LA VISTOSA""",U$S 120.000,1000 Mts,Casa,1 dormitorio,... y Punta colorada,Piriápolis,Venta,2021-12-07 00:00:00
96580,"Casa en San Francisco ""GABIMAR I""",U$S 1.600,500 Mts,Casa,2 dormitorios,... y San francisco,Piriápolis,Alquiler temporario,2021-12-07 00:00:00
96581,CASAS-VENTA-SAN-FRANCISCO-289,U$S 190.000,532 Mts,Casa,3 dormitorios,25 Cardenal y 6 Cisne y,Piriápolis,Venta,2021-12-07 00:00:00


## Limpieza de Datos

### Precio

In [302]:
dfprecio = edit.copy()
dfprecio['precio'].isna().sum() #cantidad chica de valores faltantes, estos datos deben eliminarse para seguir con el modelo

121

In [303]:
dfprecio[dfprecio['precio'].isna()==True]

,nombre_inmueble,precio,superficie,tipo,dormitorios,direccion,barrio,negocio,fecha
95473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-23 00:00:00
95479,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-23 00:00:00
95482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-23 00:00:00
95490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-23 00:00:00
95492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-23 00:00:00
...,...,...,...,...,...,...,...,...,...
96550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-07 00:00:00
96555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-07 00:00:00
96559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-07 00:00:00
96566,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-07 00:00:00


In [284]:
NaN_precios  = dfprecio['precio'].isna()
NaN_precios_loc = dropvalues[dropvalues==True].index.values
NaN_precios_loc

array([95473, 95479, 95482, 95490, 95492, 95497, 95502, 95511, 95516,
       95523, 95527, 95531, 95543, 95553, 95556, 95565, 95577, 95582,
       95602, 95604, 95614, 95618, 95620, 95640, 95647, 95654, 95665,
       95671, 95677, 95683, 95685, 95688, 95698, 95701, 95713, 95719,
       95722, 95724, 95755, 95769, 95788, 95889, 95890, 95893, 95894,
       95927, 95930, 95950, 95961, 95983, 96000, 96016, 96019, 96034,
       96043, 96045, 96047, 96081, 96087, 96089, 96096, 96097, 96101,
       96112, 96116, 96120, 96128, 96136, 96145, 96153, 96158, 96169,
       96171, 96195, 96197, 96204, 96216, 96220, 96226, 96234, 96246,
       96257, 96260, 96268, 96273, 96278, 96284, 96291, 96293, 96311,
       96322, 96329, 96343, 96350, 96352, 96365, 96370, 96377, 96384,
       96386, 96401, 96402, 96404, 96435, 96441, 96444, 96447, 96472,
       96480, 96484, 96487, 96497, 96505, 96508, 96519, 96529, 96550,
       96555, 96559, 96566, 96575], dtype=int64)

In [285]:
dfprecio.iloc[NaN_precios_loc] #son filas sin valores **IDENTIFICAR ERROR**

,nombre_inmueble,precio,superficie,tipo,dormitorios,direccion,barrio,negocio,fecha
95473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-23 00:00:00
95479,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-23 00:00:00
95482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-23 00:00:00
95490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-23 00:00:00
95492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-23 00:00:00
...,...,...,...,...,...,...,...,...,...
96550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-07 00:00:00
96555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-07 00:00:00
96559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-07 00:00:00
96566,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-07 00:00:00


**IDENTIFICAR ERROR**

In [286]:
dfprecio = dfprecio.drop(index= NaN_precios_loc)

In [287]:
dfprecio

,nombre_inmueble,precio,superficie,tipo,dormitorios,direccion,barrio,negocio,fecha
0,CASAS-ALQUILER-TEMPORAL-PUNTA-COLORADA-034,U$S 1.750,600 Mts,Casa,2 dormitorios,Av. Primera entre Rambla Costanera y 8 Guazubi...,Punta Colorada,Alquiler temporario,2021-01-03 00:00:00
1,"Alquilo Apto. 2 Dorm. Vista , 2 da Enero y Mar...",$U 800,NaN,Apartamento,2 dormitorios,Mi hijo el doctor y En Familia,Punta Del Este,Alquiler temporario,2021-01-03 00:00:00
2,"Casa en Cerro del Burro ""MIRADOR II""",U$S 100.000,300 Mts,Casa,1 dormitorio,... y Cerro del burro,Piriápolis,Venta,2021-01-03 00:00:00
3,Oficina Alquiler World Trade Center -WTC- Poci...,U$S 1.600,NaN,Oficina,NaN,... y Buceo,Buceo,Alquiler,2021-01-03 00:00:00
4,Terreno en Punta Colorada,U$S 67.000,615 Mts,Terreno,NaN,TE201282 y Punta colorada,Punta Colorada,Venta,2021-01-03 00:00:00
...,...,...,...,...,...,...,...,...,...
96578,LOCALES-COMERCIALES-ALQUILER-ANUAL-PIRIAPOLIS-...,$U 22.000,NaN,Local,NaN,Tucumán entre Sierra y Sanabria y,Piriápolis,Alquiler,2021-12-07 00:00:00
96579,"Casa en Punta Colorada ""LA VISTOSA""",U$S 120.000,1000 Mts,Casa,1 dormitorio,... y Punta colorada,Piriápolis,Venta,2021-12-07 00:00:00
96580,"Casa en San Francisco ""GABIMAR I""",U$S 1.600,500 Mts,Casa,2 dormitorios,... y San francisco,Piriápolis,Alquiler temporario,2021-12-07 00:00:00
96581,CASAS-VENTA-SAN-FRANCISCO-289,U$S 190.000,532 Mts,Casa,3 dormitorios,25 Cardenal y 6 Cisne y,Piriápolis,Venta,2021-12-07 00:00:00


In [288]:
dfprecio = dfprecio.reset_index(drop=True)
dfprecio

,nombre_inmueble,precio,superficie,tipo,dormitorios,direccion,barrio,negocio,fecha
0,CASAS-ALQUILER-TEMPORAL-PUNTA-COLORADA-034,U$S 1.750,600 Mts,Casa,2 dormitorios,Av. Primera entre Rambla Costanera y 8 Guazubi...,Punta Colorada,Alquiler temporario,2021-01-03 00:00:00
1,"Alquilo Apto. 2 Dorm. Vista , 2 da Enero y Mar...",$U 800,NaN,Apartamento,2 dormitorios,Mi hijo el doctor y En Familia,Punta Del Este,Alquiler temporario,2021-01-03 00:00:00
2,"Casa en Cerro del Burro ""MIRADOR II""",U$S 100.000,300 Mts,Casa,1 dormitorio,... y Cerro del burro,Piriápolis,Venta,2021-01-03 00:00:00
3,Oficina Alquiler World Trade Center -WTC- Poci...,U$S 1.600,NaN,Oficina,NaN,... y Buceo,Buceo,Alquiler,2021-01-03 00:00:00
4,Terreno en Punta Colorada,U$S 67.000,615 Mts,Terreno,NaN,TE201282 y Punta colorada,Punta Colorada,Venta,2021-01-03 00:00:00
...,...,...,...,...,...,...,...,...,...
96457,LOCALES-COMERCIALES-ALQUILER-ANUAL-PIRIAPOLIS-...,$U 22.000,NaN,Local,NaN,Tucumán entre Sierra y Sanabria y,Piriápolis,Alquiler,2021-12-07 00:00:00
96458,"Casa en Punta Colorada ""LA VISTOSA""",U$S 120.000,1000 Mts,Casa,1 dormitorio,... y Punta colorada,Piriápolis,Venta,2021-12-07 00:00:00
96459,"Casa en San Francisco ""GABIMAR I""",U$S 1.600,500 Mts,Casa,2 dormitorios,... y San francisco,Piriápolis,Alquiler temporario,2021-12-07 00:00:00
96460,CASAS-VENTA-SAN-FRANCISCO-289,U$S 190.000,532 Mts,Casa,3 dormitorios,25 Cardenal y 6 Cisne y,Piriápolis,Venta,2021-12-07 00:00:00


In [289]:
dfprecio['precio'] = dfprecio['precio'].replace('\$','', regex = True)
dfprecio

,nombre_inmueble,precio,superficie,tipo,dormitorios,direccion,barrio,negocio,fecha
0,CASAS-ALQUILER-TEMPORAL-PUNTA-COLORADA-034,US 1.750,600 Mts,Casa,2 dormitorios,Av. Primera entre Rambla Costanera y 8 Guazubi...,Punta Colorada,Alquiler temporario,2021-01-03 00:00:00
1,"Alquilo Apto. 2 Dorm. Vista , 2 da Enero y Mar...",U 800,NaN,Apartamento,2 dormitorios,Mi hijo el doctor y En Familia,Punta Del Este,Alquiler temporario,2021-01-03 00:00:00
2,"Casa en Cerro del Burro ""MIRADOR II""",US 100.000,300 Mts,Casa,1 dormitorio,... y Cerro del burro,Piriápolis,Venta,2021-01-03 00:00:00
3,Oficina Alquiler World Trade Center -WTC- Poci...,US 1.600,NaN,Oficina,NaN,... y Buceo,Buceo,Alquiler,2021-01-03 00:00:00
4,Terreno en Punta Colorada,US 67.000,615 Mts,Terreno,NaN,TE201282 y Punta colorada,Punta Colorada,Venta,2021-01-03 00:00:00
...,...,...,...,...,...,...,...,...,...
96457,LOCALES-COMERCIALES-ALQUILER-ANUAL-PIRIAPOLIS-...,U 22.000,NaN,Local,NaN,Tucumán entre Sierra y Sanabria y,Piriápolis,Alquiler,2021-12-07 00:00:00
96458,"Casa en Punta Colorada ""LA VISTOSA""",US 120.000,1000 Mts,Casa,1 dormitorio,... y Punta colorada,Piriápolis,Venta,2021-12-07 00:00:00
96459,"Casa en San Francisco ""GABIMAR I""",US 1.600,500 Mts,Casa,2 dormitorios,... y San francisco,Piriápolis,Alquiler temporario,2021-12-07 00:00:00
96460,CASAS-VENTA-SAN-FRANCISCO-289,US 190.000,532 Mts,Casa,3 dormitorios,25 Cardenal y 6 Cisne y,Piriápolis,Venta,2021-12-07 00:00:00


In [290]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [291]:
import yfinance as yahooFinance
from datetime import datetime, timedelta

In [292]:
def convert_currency(src, dst, amount):
    # construct the currency pair symbol
    symbol = f"{src}{dst}=X"
    FX = yahooFinance.Ticker(symbol)
    
    # extract minute data of the recent 2 days
    latest_data = FX.history(period = '1d')
    
    # get the latest datetime
    last_updated_datetime = latest_data.index[-1].to_pydatetime()
    
    # get the latest price
    latest_price = latest_data['Close'].iloc[-1]
    
    # return the latest datetime with the converted amount
    return latest_price * amount

In [294]:
convert_currency('UYU','USD',100)

2.5080256164073944

In [295]:
uyu = dfprecio['precio'].str.contains('U ', case=True, regex=True)
UYUloc = uyu[uyu==True].index.values
UYUloc

array([    1,    10,    23, ..., 96447, 96455, 96457], dtype=int64)

In [296]:
dfprecio['precio'][UYUloc] = dfprecio['precio'].iloc[UYUloc].str.replace('U','')

In [301]:
dfprecio['precio'] = dfprecio.to_string
dfprecio.dtypes

nombre_inmueble    object
precio             object
superficie         object
tipo               object
dormitorios        object
direccion          object
barrio             object
negocio            object
fecha              object
dtype: object